In [3]:
from OpenSSL import SSL, crypto

def Cliente():
    cliente_p12 = crypto.load_pkcs12(open("Cliente.p12", 'rb').read(), b'1234')
    cliente_pem = crypto.dump_certificate(crypto.FILETYPE_PEM, cliente_p12.get_certificate())
    cliente_priv = crypto.dump_privatekey(crypto.FILETYPE_PEM, cliente_p12.get_privatekey(), None,b'1234')
    cliente_pub = crypto.dump_publickey(crypto.FILETYPE_PEM, cliente_p12)
        
    with open('Cliente.pem', 'wb') as cliente_file:
        cliente_file.write(cliente_pem)
    with open('pkey_Cliente.pem', 'wb') as cliente_file:
        cliente_file.write(cliente_priv)
    with open('pubkey_Cliente.pem', 'wb') as cliente_file:
        cliente_file.write(cliente_pub)

def Servidor():
    servidor_p12 = crypto.load_pkcs12(open("Servidor.p12", 'rb').read(), b'1234')
    servidor_pem = crypto.dump_certificate(crypto.FILETYPE_PEM, servidor_p12.get_certificate())
    servidor_priv = crypto.dump_privatekey(crypto.FILETYPE_PEM, servidor_p12.get_privatekey(), None,b'1234')
    
    with open('pubkey_Servidor.pem', 'wb') as servidor_file:
        servidor_file.write(servidor_pem)
    with open('pkey_Servidor.pem', 'wb') as servidor_file:
        servidor_file.write(servidor_priv)
        
def Cert():
    cert_cer = crypto.load_certificate(crypto.FILETYPE_ASN1, open("CA.cer", 'rb').read())
    cert_pem = crypto.dump_certificate(crypto.FILETYPE_PEM, cert_cer)
    with open('CA.pem', 'wb') as cert_file:
        cert_file.write(cert_pem)
    
Cliente()
Servidor()
Cert()

TypeError: initializer for ctype 'EVP_PKEY *' must be a cdata pointer, not PKey

In [63]:
from OpenSSL import SSL, crypto
from subprocess import PIPE, Popen

def verify():
    store = crypto.X509Store()
    
    test1 = Popen(["openssl", "verify", "-CAfile", "CA.pem", "Cliente.pem"], 
                              stdin = PIPE, stdout = PIPE, stderr = PIPE)
    output1 = test1.communicate()[0]
    
    test2 = Popen(["openssl", "verify", "-CAfile", "CA.pem", "Servidor.pem"], 
                              stdin = PIPE, stdout = PIPE, stderr = PIPE)
    output2 = test2.communicate()[0]
    
    output = output1[:-1] + b' ; ' + output2[:-1]
    
    print(output)

verify()

b'Cliente.pem: OK ; Servidor.pem: OK'
